In [2]:
from requests import get
import pandas as pd
import numpy as np
import requests
import re
from tqdm import tqdm
import time

In [3]:
df = pd.read_csv('./Data/medicine_database_detailed.csv')
df.head()

,Unnamed: 0,Medicine Name,Category,Composition,Manufacturer,MRP,Uses,Side Effects
0,0,Atrest 12.5mg Tablet 10'S,ADHD,TETRABENAZINE 12.5MG,Centaur Pharmaceuticals Pvt Ltd,146.01,ATREST 12.5MG is used in the treatment of the ...,"depression, suicidal thoughts or actions, neur..."
1,1,Atrest 25mg Tablet 10'S,ADHD,TETRABENAZINE 25MG,Centaur Pharmaceuticals Pvt Ltd,261.49,ATREST 25MG is used in the treatment of the in...,"depression, suicidal thoughts or actions, neur..."
2,2,Capnea Injection 1ml,ADHD,Caffeine Citrate 20 MG,Cipla Ltd,272.25,It is used in the treatment of interrupted bre...,serious intestinal disease with bloody stools ...
3,3,COGNIX 120mg Tablet 10's,ADHD,GINKGO BILOBA 120 mg,Eris Lifesciences Ltd,128.70,Data Not Found,Data Not Found
4,4,A CN Gel(Topical) 20gm,Acne,CLINDAMYCIN 1%+NICOTINAMIDE 4%,Eskon Pharma,135.00,mild to moderate acne (spots),"dry skin, redness of the skin, skin burning or..."


In [6]:
df['Medicine Name']

0             Atrest 12.5mg Tablet 10'S
1               Atrest 25mg Tablet 10'S
2                  Capnea Injection 1ml
3              COGNIX 120mg Tablet 10's
4                A CN Gel(Topical) 20gm
                      ...              
23561          Wound Fix Solution 100ml
23562             Wounsol Ointment 15gm
23563          ZEROSEPTIC Ointment 15gm
23564    DS GOLD ANTI AGEING Cream 50gm
23565                    Versa Gel 30gm
Name: Medicine Name, Length: 23566, dtype: object

In [50]:
df['Category'].unique()

array(['ADHD', 'Acne', 'Alcohol', 'Allergies', 'Alzheimer', 'Amoebiasis',
       'Anaemia', 'Anaesthesia', 'Angina', 'Anxiety', 'Appetite',
       'Arrhythmiasis', 'Arthritis', 'Asthma/COPD', 'Auto', 'Ayurvedic',
       'Bladder', 'Bleeding', 'Blood', 'Bone', 'Burns', 'Cancer/Oncology',
       'Cholelithiasis/Gall', 'Cleanser', 'Constipation', 'Contraception',
       'Cough', 'Crack', 'Dandruff', 'Depression', 'Diabetes',
       'Diagnostic', 'Diarrhoea', 'Digestion', 'Dry', 'Ear',
       'Epilepsy/Convulsion', 'Eye', 'Fever', 'Fungal', 'Gastro',
       'General', 'Glaucoma', 'Gout', 'Haematopoiesis', 'Haemorrhoid',
       'Hair', 'Head', 'Heart', 'Hepatitis', 'High', 'Hormonal',
       'Hyperpigmentation', 'Hypertension', 'Hyperthyroidism', 'Hypnosis',
       'Hypotension', 'Hypothyroidism', 'Idiopathic', 'Immune',
       'Immunoglobulins', 'Impotence/Erectile', 'Infection', 'Kidney',
       'Leprosy', 'Liver', 'Lubrication', 'Malarial', 'Migraine',
       'Muscle', 'Mydriasis', 'Napp

In [65]:
TYPE = ['Tablet', 'Injection', 'Ointment', 'Cream', 'Gel', 'Solution', 'Syrup', 'Device', 'Infusion']
STRENGTH = ['mg', 'gm', 'ml']
QUANTITY = ['10\'S']

In [9]:
from fuzzywuzzy import process

In [71]:
nameList = []
strengthList = []
typeList = []
quantityList = []
quantityTypeList = []

for medicine in tqdm(df["Medicine Name"]):
    medicineList = medicine.split()

    type = process.extract(medicine, TYPE, limit=1)[0][0]
    medicineList.remove(process.extract(type, medicineList, limit=1)[0][0])
    typeList.append(type)
    if type in ['Device', 'Infusion']:
        strengthList.append(None)
        quantityTypeList.append(strength)
        quantityList.append(1)
        nameList.append(' '.join(medicineList))
        continue

    quantityType = process.extract(medicine, STRENGTH, limit=1)[0][0]
    strength = process.extract(quantityType, medicineList, limit=1)[0][0]
    medicineList.remove(strength)
    strength = ''.join(re.findall(r'\d+', strength))
    quantityTypeList.append(quantityType)
    strengthList.append(strength)

    quantity = process.extract(medicine, QUANTITY, limit=1)[0][0]
    try:
        _quantity = process.extract(quantity, medicineList, limit=1)[0][0]
        medicineList[::-1].remove(_quantity)
    except:
        _quantity = '1'
    if '10\'S' in medicineList:
        medicineList.remove('10\'S')

    if '10\'s' in medicineList:
        medicineList.remove('10\'s')

    if '%' not in _quantity:
        _quantity = ''.join(re.findall(r'\d+', _quantity))
        quantityList.append(_quantity)
    else:
        quantityList.append(1)


    nameList.append(' '.join(medicineList))

100%|███████████████████████████████████████████████████████████████████████████| 23566/23566 [00:58<00:00, 404.22it/s]


In [67]:
df['Medicine Name'][12482]

'Decapeptyl Injection(Depot)'

In [75]:
clean_data = pd.DataFrame()
clean_data['name'] = nameList
clean_data['type'] = typeList
clean_data['strength'] = strengthList
clean_data['quantityType'] = quantityTypeList
clean_data['quantity'] = quantityList
clean_data['medicine_name'] = df['Medicine Name']
clean_data['category'] = df['Category']
clean_data['composition'] = df['Composition']
clean_data['manufacturer'] = df['Manufacturer']
clean_data['mrp'] = df['MRP']
clean_data['uses'] = df['Uses']
clean_data['side_effects'] = df['Side Effects']
clean_data

,name,type,strength,quantityType,quantity,medicine_name,category,composition,manufacturer,mrp,uses,side_effects
0,Atrest,Tablet,125,mg,10,Atrest 12.5mg Tablet 10'S,ADHD,TETRABENAZINE 12.5MG,Centaur Pharmaceuticals Pvt Ltd,146.01,ATREST 12.5MG is used in the treatment of the ...,"depression, suicidal thoughts or actions, neur..."
1,Atrest,Tablet,25,mg,10,Atrest 25mg Tablet 10'S,ADHD,TETRABENAZINE 25MG,Centaur Pharmaceuticals Pvt Ltd,261.49,ATREST 25MG is used in the treatment of the in...,"depression, suicidal thoughts or actions, neur..."
2,Capnea,Injection,1,ml,,Capnea Injection 1ml,ADHD,Caffeine Citrate 20 MG,Cipla Ltd,272.25,It is used in the treatment of interrupted bre...,serious intestinal disease with bloody stools ...
3,COGNIX,Tablet,120,mg,10,COGNIX 120mg Tablet 10's,ADHD,GINKGO BILOBA 120 mg,Eris Lifesciences Ltd,128.70,Data Not Found,Data Not Found
4,A CN,Gel,20,gm,,A CN Gel(Topical) 20gm,Acne,CLINDAMYCIN 1%+NICOTINAMIDE 4%,Eskon Pharma,135.00,mild to moderate acne (spots),"dry skin, redness of the skin, skin burning or..."
...,...,...,...,...,...,...,...,...,...,...,...,...
23561,Wound Fix,Solution,100,ml,,Wound Fix Solution 100ml,Wound,HYPOCHLOROUS ACID 0.006%+OXIDIZED WATER 99.97%,Strides Shasun Ltd,140.00,Data Not Found,Data Not Found
23562,Wounsol,Ointment,15,gm,,Wounsol Ointment 15gm,Wound,Povidone Iodine+Sucralfate,Theia Health Care Pvt Ltd,75.00,Data Not Found,Data Not Found
23563,ZEROSEPTIC,Ointment,15,gm,,ZEROSEPTIC Ointment 15gm,Wound,POVIDINE IODINE 5 %,Helios Pharmaceuticals,75.00,Data Not Found,Data Not Found
23564,DS GOLD ANTI AGEING,Cream,50,gm,,DS GOLD ANTI AGEING Cream 50gm,Wrinkle/Anti,SKINCARE PREPARATION,Adonis Laboratories Pvt Ltd,999.00,Data Not Found,Data Not Found


In [85]:
clean_data.isnull().sum()

name             0
type             0
strength         0
quantityType     0
quantity         0
medicine_name    0
category         0
composition      0
manufacturer     0
mrp              0
uses             0
side_effects     0
dtype: int64

In [87]:
clean_data['strength'] = clean_data['strength'].fillna(0)
clean_data['strength'] = clean_data['strength'].replace([''],[0])
clean_data['quantity'] = clean_data['quantity'].replace([''],[0])
clean_data['uses'] = clean_data['uses'].replace(['Data Not Found'],['NA'])
clean_data['side_effects'] = clean_data['side_effects'].replace(['Data Not Found'],['NA'])
clean_data['uses'] = clean_data['uses'].fillna('NA')
clean_data['side_effects'] = clean_data['side_effects'].fillna('NA')
clean_data

,name,type,strength,quantityType,quantity,medicine_name,category,composition,manufacturer,mrp,uses,side_effects
0,Atrest,Tablet,125,mg,10,Atrest 12.5mg Tablet 10'S,ADHD,TETRABENAZINE 12.5MG,Centaur Pharmaceuticals Pvt Ltd,146.01,ATREST 12.5MG is used in the treatment of the ...,"depression, suicidal thoughts or actions, neur..."
1,Atrest,Tablet,25,mg,10,Atrest 25mg Tablet 10'S,ADHD,TETRABENAZINE 25MG,Centaur Pharmaceuticals Pvt Ltd,261.49,ATREST 25MG is used in the treatment of the in...,"depression, suicidal thoughts or actions, neur..."
2,Capnea,Injection,1,ml,0,Capnea Injection 1ml,ADHD,Caffeine Citrate 20 MG,Cipla Ltd,272.25,It is used in the treatment of interrupted bre...,serious intestinal disease with bloody stools ...
3,COGNIX,Tablet,120,mg,10,COGNIX 120mg Tablet 10's,ADHD,GINKGO BILOBA 120 mg,Eris Lifesciences Ltd,128.70,NA,NA
4,A CN,Gel,20,gm,0,A CN Gel(Topical) 20gm,Acne,CLINDAMYCIN 1%+NICOTINAMIDE 4%,Eskon Pharma,135.00,mild to moderate acne (spots),"dry skin, redness of the skin, skin burning or..."
...,...,...,...,...,...,...,...,...,...,...,...,...
23561,Wound Fix,Solution,100,ml,0,Wound Fix Solution 100ml,Wound,HYPOCHLOROUS ACID 0.006%+OXIDIZED WATER 99.97%,Strides Shasun Ltd,140.00,NA,NA
23562,Wounsol,Ointment,15,gm,0,Wounsol Ointment 15gm,Wound,Povidone Iodine+Sucralfate,Theia Health Care Pvt Ltd,75.00,NA,NA
23563,ZEROSEPTIC,Ointment,15,gm,0,ZEROSEPTIC Ointment 15gm,Wound,POVIDINE IODINE 5 %,Helios Pharmaceuticals,75.00,NA,NA
23564,DS GOLD ANTI AGEING,Cream,50,gm,0,DS GOLD ANTI AGEING Cream 50gm,Wrinkle/Anti,SKINCARE PREPARATION,Adonis Laboratories Pvt Ltd,999.00,NA,NA


In [88]:
clean_data.to_csv('./Data/medicine_data.csv')

In [97]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23566 entries, 0 to 23565
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           23566 non-null  object
 1   type           23566 non-null  object
 2   strength       23566 non-null  object
 3   quantityType   23566 non-null  object
 4   quantity       23566 non-null  object
 5   medicine_name  23566 non-null  object
 6   category       23566 non-null  object
 7   composition    23566 non-null  object
 8   manufacturer   23566 non-null  object
 9   mrp            23566 non-null  object
 10  uses           23566 non-null  object
 11  side_effects   23566 non-null  object
dtypes: object(12)
memory usage: 2.2+ MB


"WINCIT 5mg Tablet 10's"

In [114]:
import pandas as pd
import requests
import json
import random
from tqdm import tqdm
import time

df = pd.read_csv('./Data/medicine_data.csv')

index = random.sample(range(0,len(df)), 10000)
for i in tqdm(index):
    mrp = df["mrp"][i].split('.')[0]
    mrp = re.sub(r'[^\w\s]', '', mrp)
    body = {
      "category": df["category"][i],
      "composition": df["composition"][i],
      "manufacturer": df["manufacturer"][i],
      "medicineFullName": df["medicine_name"][i],
      "medicineName": df["name"][i],
      "medicineQuantityType": df["quantityType"][i] if df["quantityType"][i] in ['mg', 'ml', 'gm'] else 'none',
      "mrp": float(mrp),
      "quantity": int(df["quantity"][i]),
      "sideEffect": df["side_effects"][i],
      "strength": str(df["strength"][i]),
      "type": df["type"][i],
      "uses": df["uses"][i]
    }
    body = json.dumps(body)
    headers = {
        'Content-Type': 'application/json'
    }
    url = 'http://20.198.81.29:8080/medicine'
    x = requests.post(url, data=body, headers=headers)
    

 22%|█████████████████                                                            | 2223/10000 [04:22<15:17,  8.48it/s]


KeyboardInterrupt: 